Classifying Flowers using Transfer Learning in Keras

1- Download a small flower dataset (http://download.tensorflow.org/example_images/flower_photos.tgz). This dataset has 5 classes (Daisy, Dandelion, Rese, Sunflower, and Tulip). Images for each class are stored in its own folder.

2- The images have different dimensions. Resize all of them to 150x150.

3- Split images to 75-25% for training and test. Make sure you have the same distribution of flower types between train and test datasets. 

4- Use a VGG16 model (pre-trained on ImageNet)

5- Remove the top layers (fully connected layers)

6- Add your own fully connected layers (one with 256 nodes using ‘relu’ activation and output layer with 5 nodes and ‘softmax’ activation)

7- First, freeze all layers of VGG16, train (fine-tune) and evaluate the model. You need to pick the right hyper-parameters for your training (try with different ones)

8- Second, unfreeze the last block of VGG16 (block5), re-train and evaluate the model

9- Unfreeze all the layers and try again. 

10- Compare the accuracy you got in both cases . Which one is better and why?

In [1]:
import tensorflow as tf

In [25]:
import requests

url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

# data_preparation

In [2]:

from sklearn.model_selection import train_test_split

%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.applications.resnet50 import preprocess_input
  
# #'''''' Do not use GPU '''''''
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# #'''''' Do not use GPU '''''''


train_dir = "flower_photos/"
# validation_dir = "flower_photos/validation"

batch_size=32
img_size=150

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary', shuffle=True,
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary', shuffle=True,
    subset='validation') # set as validation data

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

for data_batch, labels_batch in validation_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
    
    
# model.fit_generator(
#     train_generator,
#     steps_per_epoch = train_generator.samples // batch_size,
#     validation_data = validation_generator, 
#     validation_steps = validation_generator.samples // batch_size,
#     epochs = nb_epochs)

Found 2755 images belonging to 5 classes.
Found 916 images belonging to 5 classes.
data batch shape: (32, 150, 150, 3)
labels batch shape: (32,)
data batch shape: (32, 150, 150, 3)
labels batch shape: (32,)


# model_1

In [3]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_size, img_size, 3),
    pooling=None,
    classes=1000,
)
x = base_model.output
x = Flatten()(x)
x = Dense(256, name="new_dense", activation="relu")(x)
predictions = Dense(5, name="predictions", activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

freeze_layers_from = len(base_model.layers)
print("Freezing from layer 0 to " + str(freeze_layers_from))
for layer in model.layers[:freeze_layers_from]:
    layer.trainable = False
for layer in model.layers[freeze_layers_from:]:
    layer.trainable = True
# for i, layer in enumerate(model.layers):
#     print(i, layer.name + ' trainable=' , layer.trainable )

model.compile(
    loss="categorical_crossentropy", optimizer=RMSprop(lr=0.0001), metrics=["accuracy"]
)
checkpointer = ModelCheckpoint(
    "best.hdf5", monitor="val_loss", mode="auto", save_best_only=True, verbose=1
)
earlystopper = EarlyStopping(monitor="val_loss", patience=10, verbose=1)
callbackser = [earlystopper, checkpointer]

model.summary()

Freezing from layer 0 to 19
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (Non

In [45]:
history = model.fit_generator(
    train_generator,
    epochs=3,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbackser,
    verbose=1,
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 86 steps, validate for 28 steps
Epoch 1/3
85/86 [============================>.] - ETA: 11s - loss: 28.2657 - accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 33.98722, saving model to best.hdf5
86/86 [==============================] - 1304s 15s/step - loss: 28.3945 - accuracy: 0.0000e+00 - val_loss: 33.9872 - val_accuracy: 0.0056
Epoch 2/3
85/86 [============================>.] - ETA: 11s - loss: 42.3312 - accuracy: 0.0123
Epoch 00002: val_loss did not improve from 33.98722
86/86 [==============================] - 1306s 15s/step - loss: 42.3118 - accuracy: 0.0125 - val_loss: 49.8096 - val_accuracy: 0.0134
Epoch 3/3
85/86 [============================>.] - ETA: 11s - loss: 52.5565 - accuracy: 0.0201
Epoch 00003: val_loss did not improve from 33.98722
86/86 [==============================] - 1296s 15s/step - loss: 52.6450 - accuracy: 0.0198 - val_loss: 52.0536 - val_accuracy: 0.0346


# model_2

In [7]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_size, img_size, 3),
    pooling=None,
    classes=1000,
)
x = base_model.output
x = Flatten()(x)
x = Dense(256, name="new_dense", activation="relu")(x)
predictions = Dense(5, name="predictions", activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

freeze_layers_from = len(base_model.layers)
print("Freezing from layer 0 to " + str(freeze_layers_from))
for layer in model.layers[:freeze_layers_from]:
    if "block5" in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False
for layer in model.layers[freeze_layers_from:]:
    layer.trainable = True
# for i, layer in enumerate(model.layers):
#     print(i, layer.name + ' trainable=' , layer.trainable )

model.compile(
    loss="categorical_crossentropy", optimizer=RMSprop(lr=0.0001), metrics=["accuracy"]
)
checkpointer = ModelCheckpoint(
    "best.hdf5", monitor="val_loss", mode="auto", save_best_only=True, verbose=1
)
earlystopper = EarlyStopping(monitor="val_loss", patience=10, verbose=1)
callbackser = [earlystopper, checkpointer]

model.summary()

Freezing from layer 0 to 19
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (N

In [8]:
history = model.fit_generator(
    train_generator,
    epochs=3,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbackser,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 86 steps, validate for 28 steps
Epoch 1/3
85/86 [============================>.] - ETA: 14s - loss: 6749992.4296 - accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 30783608.21429, saving model to best.hdf5
86/86 [==============================] - 1611s 19s/step - loss: 6973926.5113 - accuracy: 0.0000e+00 - val_loss: 30783608.2143 - val_accuracy: 0.0000e+00
Epoch 2/3
85/86 [============================>.] - ETA: 14s - loss: 119074071.9529 - accuracy: 0.0000e+00
Epoch 00002: val_loss did not improve from 30783608.21429
86/86 [==============================] - 1548s 18s/step - loss: 121368816.1183 - accuracy: 0.0000e+00 - val_loss: 265970922.2857 - val_accuracy: 0.0000e+00
Epoch 3/3
 4/86 [>.............................] - ETA: 16:27 - loss: 209424437.3333 - accuracy: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_lo

KeyboardInterrupt: 

# model_3

In [9]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_size, img_size, 3),
    pooling=None,
    classes=1000,
)
x = base_model.output
x = Flatten()(x)
x = Dense(256, name="new_dense", activation="relu")(x)
predictions = Dense(5, name="predictions", activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

freeze_layers_from = len(base_model.layers)
print("Freezing from layer 0 to " + str(freeze_layers_from))
for layer in model.layers:
    layer.trainable = True
# for i, layer in enumerate(model.layers):
#     print(i, layer.name + ' trainable=' , layer.trainable )

model.compile(
    loss="categorical_crossentropy", optimizer=RMSprop(lr=0.0001), metrics=["accuracy"]
)
checkpointer = ModelCheckpoint(
    "best.hdf5", monitor="val_loss", mode="auto", save_best_only=True, verbose=1
)
earlystopper = EarlyStopping(monitor="val_loss", patience=10, verbose=1)
callbackser = [earlystopper, checkpointer]

model.summary()

Freezing from layer 0 to 19
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (N

In [10]:
history = model.fit_generator(
    train_generator,
    epochs=3,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbackser,
    verbose=1,
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 86 steps, validate for 28 steps
Epoch 1/3
29/86 [=========>....................] - ETA: 40:09 - loss: 45656268192482.7031 - accuracy: 0.0000e+00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


KeyboardInterrupt: 

# comparison

Due to limitations of my local machine I was only able to run experiments 2 and 3 for a number of steps.

Performance was in the order of Model1 > Model2 > Model3 (which you can infer by comparing the epoch 1 losses, and val accuracy in models 1, 2), considering the training window.

Side note :: Perhaps the usage of relu, a lack of regularization also caused the exploding gradients in Model2, Model3.

Why? Since the new dataset is smaller compared to the initial volume its trained on, it is not a good idea to fine tune it. It would be best to use the base model as a feature extractor as we did in the case of Model 1, or considering maintaining just the first few layers and training a linear classifier on it.